<a href="https://colab.research.google.com/github/konstantins7/rabota1/blob/main/vitrina__200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install google-auth requests pandas google-api-python-client beautifulsoup4 lxml

import requests
import pandas as pd
import datetime
import os
import urllib.request
from bs4 import BeautifulSoup
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession
import time

# Настройки
DAILY_QUOTA = 200  # Пример дневной квоты, замените на ваше значение
INDEXING_URL = "https://indexing.googleapis.com/v3/urlNotifications:publish"
LONG_OPERATION_THRESHOLD = 300  # Порог времени для длительной операции (в секундах)

# Пути к файлам
VITRINA_KEY_FILE_LOCATION = '/content/drive/MyDrive/Colab Notebooks/vitrina24kz-82fd975fbbe4.json'
VITRINA_TXT_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/vitrina24_urls.txt'
VITRINA_PROGRESS_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/vitrina24_indexing_progress.csv'
VITRINA_SITEMAP_URL = 'https://vitrina24.kz/sitemap.xml'
TELEGRAM_TOKEN_FILE = '/content/drive/MyDrive/Colab Notebooks/tgtokenvitrina24kz.txt'
TELEGRAM_CHAT_ID = '-1002207415181'  # chat_id для Telegram канала

# Области доступа OAuth
SCOPES = ["https://www.googleapis.com/auth/indexing"]

# Функция для загрузки токена из файла
def load_telegram_token(token_file):
    with open(token_file, 'r') as file:
        token = file.read().strip()
    return token

# Функция для отправки сообщений в Telegram
def send_telegram_message(token, chat_id, message):
    url = f'https://api.telegram.org/bot{token}/sendMessage'
    data = {'chat_id': chat_id, 'text': message}
    response = requests.post(url, data=data)
    if response.status_code != 200:
        print(f"Ошибка отправки сообщения в Telegram: {response.status_code} - {response.text}")
    return response.json()

# Функция для логирования сообщений
def log_message(message):
    print(message)  # Вывод сообщения для отладки
    with open("/content/drive/MyDrive/Colab Notebooks/indexing_log.txt", "a") as log_file:
        log_file.write(f"{datetime.datetime.now()}: {message}\n")

# Функция для проверки наличия файлов
def check_file_exists(file_path):
    if not os.path.exists(file_path):
        log_message(f"Ошибка: Файл {file_path} не найден.")
        return False
    return True

# Функция для получения URL-адресов из sitemap
def get_urls_from_sitemap(sitemap_url):
    sitemap_urls = []
    response = urllib.request.urlopen(sitemap_url)
    soup = BeautifulSoup(response, 'xml')
    for loc in soup.find_all('loc'):
        sitemap_urls.append(loc.text)
    return sitemap_urls

# Функция для отправки URL на индексацию
def publish_url(url, authed_session):
    data = {
        "url": url,
        "type": "URL_UPDATED"
    }
    response = authed_session.post(INDEXING_URL, json=data)
    return response

# Функция для индексации URL-адресов
def index_urls(key_file_location, txt_file_path, progress_file_path, sitemap_url, domain_url, site_name):
    # Проверка наличия ключевого файла
    if not check_file_exists(key_file_location):
        return
    # Загрузка токена и chat_id для Telegram
    telegram_token = load_telegram_token(TELEGRAM_TOKEN_FILE)
    telegram_chat_id = TELEGRAM_CHAT_ID
    # Отправка сообщения о начале индексации
    send_telegram_message(telegram_token, telegram_chat_id, f"Начата индексация для {site_name}.")
    # Создание учетных данных и авторизованного сеанса
    credentials = service_account.Credentials.from_service_account_file(key_file_location, scopes=SCOPES)
    authed_session = AuthorizedSession(credentials)
    log_message("Учетные данные успешно созданы и сессия авторизована.")
    # Загрузка данных из TXT файла
    if check_file_exists(txt_file_path):
        with open(txt_file_path, 'r') as file:
            urls = [line.strip() for line in file]
        log_message(f"Файл {txt_file_path} успешно прочитан.")
    else:
        urls = get_urls_from_sitemap(sitemap_url)
        with open(txt_file_path, 'w') as file:
            for url in urls:
                file.write(f"{url}\n")
        log_message(f"Создан новый файл {txt_file_path} с URL-адресами из sitemap.")
    # Создание или загрузка файла прогресса
    try:
        progress = pd.read_csv(progress_file_path)
    except FileNotFoundError:
        progress = pd.DataFrame(urls, columns=['Product URL'])
        progress['Indexed'] = False
        progress.to_csv(progress_file_path, index=False)
        log_message(f"Создан новый файл прогресса: {progress_file_path}")
    # Проверка квоты перед началом индексации
    try:
        response = authed_session.get(f"https://indexing.googleapis.com/v3/urlNotifications/metadata?url={domain_url}")
        quota_info = response.json()
        log_message(f"Информация о квоте: {quota_info}")
    except Exception as e:
        log_message(f"Ошибка при проверке квоты: {e}")
        send_telegram_message(telegram_token, telegram_chat_id, f"Ошибка при проверке квоты для {site_name}: {e}")
        return
    # Отслеживание количества отправленных URL
    urls_sent_today = 0
    start_time = time.time()
    for index, row in progress.iterrows():
        if urls_sent_today >= DAILY_QUOTA:
            log_message("Достигнута дневная квота. Попробуйте снова завтра.")
            send_telegram_message(telegram_token, telegram_chat_id, f"Достигнута дневная квота для {site_name}. Попробуйте снова завтра.")
            break
        if not row['Indexed']:
            url = row['Product URL']
            response = publish_url(url, authed_session)
            if response.status_code == 200:
                progress.at[index, 'Indexed'] = True
                urls_sent_today += 1
                log_message(f"URL отправлен успешно: {url}")
            elif response.status_code == 429:
                log_message("Достигнута дневная квота. Попробуйте снова завтра.")
                send_telegram_message(telegram_token, telegram_chat_id, f"Достигнута дневная квота для {site_name}. Попробуйте снова завтра.")
                break
            elif response.status_code == 403:
                log_message(f"Ошибка при отправке {url}: {response.status_code} {response.json()}")
                send_telegram_message(telegram_token, telegram_chat_id, f"Ошибка при отправке {url} для {site_name}: {response.status_code} {response.json()}")
                break
            else:
                log_message(f"Ошибка при отправке {url}: {response.status_code} {response.json()}")
                send_telegram_message(telegram_token, telegram_chat_id, f"Ошибка при отправке {url} для {site_name}: {response.status_code} {response.json()}")
                break
        elapsed_time = time.time() - start_time
        if elapsed_time > LONG_OPERATION_THRESHOLD:
            send_telegram_message(telegram_token, telegram_chat_id, f"Индексация для {site_name} выполняется дольше обычного. Время выполнения: {elapsed_time:.2f} секунд.")
    progress.to_csv(progress_file_path, index=False)
    log_message(f"Сегодня было отправлено {urls_sent_today} URL-адресов для {site_name}.")
    send_telegram_message(telegram_token, telegram_chat_id, f"Сегодня было отправлено {urls_sent_today} URL-адресов для {site_name}. Индексация завершена.")

# Выполнение индексации для vitrina24.kz
index_urls(
    VITRINA_KEY_FILE_LOCATION,
    VITRINA_TXT_FILE_PATH,
    VITRINA_PROGRESS_FILE_PATH,
    VITRINA_SITEMAP_URL,
    "https://vitrina24.kz",
    "vitrina24.kz"
)


Учетные данные успешно созданы и сессия авторизована.
Файл /content/drive/MyDrive/Colab Notebooks/vitrina24_urls.txt успешно прочитан.
Информация о квоте: {'url': 'https://vitrina24.kz', 'latestUpdate': {'url': 'https://vitrina24.kz', 'type': 'URL_UPDATED', 'notifyTime': '2024-06-24T07:10:51.357812412Z'}}
Достигнута дневная квота. Попробуйте снова завтра.
Сегодня было отправлено 0 URL-адресов для vitrina24.kz.


In [23]:
import requests

TOKEN = "7415311923:AAGV1tQ7tr8FTvWNuEGsX-fwvKCfYvPiBJ0"
channel_username = "@indexnowmedvitrina24kz"  # Замените на имя вашего канала

url = f"https://api.telegram.org/bot{TOKEN}/sendMessage"
params = {
    "chat_id": channel_username,
    "text": "Test message"
}

response = requests.post(url, params=params)
print(response.json())


{'ok': True, 'result': {'message_id': 3, 'sender_chat': {'id': -1002207415181, 'title': 'Index Vitrina24', 'username': 'indexnowmedvitrina24kz', 'type': 'channel'}, 'chat': {'id': -1002207415181, 'title': 'Index Vitrina24', 'username': 'indexnowmedvitrina24kz', 'type': 'channel'}, 'date': 1719224854, 'text': 'Test message'}}


In [26]:
import requests

TELEGRAM_BOT_TOKEN = '7415311923:AAGV1tQ7tr8FTvWNuEGsX-fwvKCfYvPiBJ0'
CHANNEL_USERNAME = '@indexnowmedvitrina24kz'  # Замените на ваше имя канала
MESSAGE = 'Это тестовое сообщение для получения ID канала.'

url = f'https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage'
params = {
    'chat_id': CHANNEL_USERNAME,
    'text': MESSAGE
}

response = requests.post(url, params=params)
data = response.json()

print(data)  # Посмотреть полный ответ

# Извлечение chat_id из ответа
chat_id = data['result']['chat']['id']
print(f"chat_id: {chat_id}")


{'ok': True, 'result': {'message_id': 4, 'sender_chat': {'id': -1002207415181, 'title': 'Index Vitrina24', 'username': 'indexnowmedvitrina24kz', 'type': 'channel'}, 'chat': {'id': -1002207415181, 'title': 'Index Vitrina24', 'username': 'indexnowmedvitrina24kz', 'type': 'channel'}, 'date': 1719225354, 'text': 'Это тестовое сообщение для получения ID канала.'}}
chat_id: -1002207415181
